## City weather condition: Openweathermap API and Citipy

In [ ]:
import pandas as pd
import numpy as np
import csv
import os
import random
import time
import requests as rq
from citipy import citipy
import openweathermapy.core as ow
import matplotlib.pyplot as plt
import seaborn as sns

os.system('touch weather500.csv')

api_key = '5a67f7dc345de96a6433433dd50f1261'
url = 'http://api.openweathermap.org/data/2.5/weather?'
units = 'imperial'

query_url = url + "appid=" + api_key + "&units=" + units + "&q="

# Building dataframe structure

columns = ['lat','lon','City','Country','Temp (F)','Humidity (%)','Cloudiness (%)','Wind_Speed (mph)','Latitude','Longitude']
index = np.arange(2000)
coor_df = pd.DataFrame(columns=columns, index=index)

# Randomly generating unique latitide and longtitude coordinates.

seen = set()

for index, row in coor_df.iterrows():
    x,y = random.randint(-90, 90),random.randint(-180, 180)
    while len(seen) < 500:
        while (x, y) in seen:
            x,y = random.randint(-90, 90),random.randint(-180, 180)
        seen.add((x, y))
    coor_df.loc[index,'lat'] = x
    coor_df.loc[index,'lon'] = y


In [ ]:
coor_df.head()

In [ ]:
for index, row in coor_df.iterrows():
    city = citipy.nearest_city(coor_df.loc[index,'lat'],coor_df.loc[index,'lon'])
    coor_df.loc[index,'City'] = city.city_name
    coor_df.loc[index,'Country'] = city.country_code

In [ ]:
coor_df.head()

In [ ]:
citydf = coor_df.drop_duplicates(subset='City')
citydf.head()

In [ ]:
len(citydf['City'])

In [ ]:
api_log = []

for index, row in citydf.iterrows(): 
    weather_api = query_url + citydf.loc[index,'City'] 
    weather_data = rq.get(weather_api).json()
    api_log.append({'URL for City': weather_api})
    
    temp_data = weather_data['main']['temp']
    humid_data = weather_data['main']['humidity']
    cloud_data = weather_data['clouds']['all']
    wind_data = weather_data['wind']['speed']
    lat_data = weather_data['coord']['lat']
    lon_data = weather_data['coord']['lon']
    
    citydf.set_value(index,'Temp (F)',temp_data)
    citydf.set_value(index,'Humidity', humid_data)
    citydf.set_value(index,'Cloudiness', cloud_data)
    citydf.set_value(index,'Wind_Speed', wind_data)
    citydf.set_value(index,'Latitude', lat_data)
    citydf.set_value(index,'Longitude', lon_data)
    
    time.sleep(5)


In [ ]:
citydf.head()

In [ ]:
api_log

In [ ]:
citydf.to_csv('weather500.csv')

In [ ]:
# Scatter plot for Temp vs Latitude
plt.scatter(citydf['Latitude'],citydf['Temp (F)'])
plt.title('City Temperature (F) vs. City Latitude')
plt.xlabel('Latitude')
plt.ylabel('Temperature (F)')
plt.xlim([-90,90])
# plt.ylim([-180,180])

plt.savefig('Temp vs Lat.png')

plt.show()

In [ ]:
# Scatter plot for Humidity vs Latitude
plt.scatter(citydf['Latitude'],citydf['Humidity'])
plt.title('City Humidity vs. City Latitude')
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.xlim([-90,90])
# plt.ylim([-180,180])

plt.savefig('Humidity vs Lat.png')

plt.show()

In [ ]:
# Scatter plot for Cloudiness vs Latitude
plt.scatter(citydf['Latitude'],citydf['Cloudiness'])
plt.title('City Cloudiness vs. City Latitude')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.xlim([-90,90])
# plt.ylim([-180,180])

plt.savefig('Cloudiness vs Lat.png')

plt.show()

In [ ]:
# Scatter plot for Wind speed vs Latitude
plt.scatter(citydf['Latitude'],citydf['Wind_Speed'])
plt.title('City Wind Speed vs. City Latitude')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.xlim([-90,90])
# plt.ylim([-180,180])

plt.savefig('Wind Speed vs Lat.png')

plt.show()